In [1]:
import os
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

class_names = ['0', '1', '2', '3', 
               '4', '5', '6', '7', 
               '8', '9', 'A', 'B', 
               'C', 'D', 'E', 'F', 
               'G', 'H', 'K', 'L', 
               'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 
               'V', 'X', 'Y', 'Z']

datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='validation',
    class_mode='sparse'
)

Found 1729 images belonging to 32 classes.
Found 417 images belonging to 32 classes.


In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (5,5), activation='elu', 
                           padding='same',
                           input_shape=(28, 12, 3)),
    tf.keras.layers.Conv2D(64, (5,5), activation='elu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='elu', 
                           padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='elu',
                           padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='elu',
                           padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(42, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(84, activation='elu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='softmax')
])

2022-01-07 00:30:57.052872: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-07 00:30:57.053190: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 12, 64)        4864      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 12, 64)        102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 6, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 6, 128)        73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 6, 128)        147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 6, 128)        147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 3, 128)         0

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0000003),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
class stopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('accuracy') > 0.99) :
            print('\nReached 99% accuracy so stopping training')
            self.model.stop_training = True

In [6]:
callback = stopCallback()
model.fit(train_generator, epochs=1000, callbacks=[callback])

Train for 87 steps
Epoch 1/1000
87/87 [==============================] - 30s 348ms/step - loss: 3.4644 - accuracy: 0.0283
Epoch 2/1000
87/87 [==============================] - 23s 266ms/step - loss: 3.4653 - accuracy: 0.0231
Epoch 3/1000
87/87 [==============================] - 23s 263ms/step - loss: 3.4633 - accuracy: 0.0411
Epoch 4/1000
87/87 [==============================] - 23s 268ms/step - loss: 3.4638 - accuracy: 0.0295
Epoch 5/1000
87/87 [==============================] - 22s 249ms/step - loss: 3.4611 - accuracy: 0.0463
Epoch 6/1000
87/87 [==============================] - 22s 256ms/step - loss: 3.4617 - accuracy: 0.0335
Epoch 7/1000
87/87 [==============================] - 22s 254ms/step - loss: 3.4605 - accuracy: 0.0411
Epoch 8/1000
87/87 [==============================] - 22s 254ms/step - loss: 3.4610 - accuracy: 0.0411
Epoch 9/1000
87/87 [==============================] - 22s 255ms/step - loss: 3.4603 - accuracy: 0.0388
Epoch 10/1000
87/87 [==============================] -

In [ ]:
model.evaluate(validation_generator, verbose=1)

In [ ]:
path1 = "../Testing_set/greenparkingchar"
path2 = "../Testing_set/greenparking2char"

all_path = [path1, path2]

final = "../charLabel/"

i = 1

for path in all_path:
    for file in os.listdir(path) :
        image_mp= mpimg.imread(path + '/' + file) / 255
        image_mp = np.expand_dims(image_mp, axis=0)
        shutil.copyfile(path + '/' + file, final + class_names[np.argmax(model.predict(image_mp))] + '/' + file)
        i += 1
        if i == 10:
            break
    if i == 10:
        break